https://www.kaggle.com/code/pablo94/hidden-risk-factors-got-by-2nd-poly-degree

https://www.kaggle.com/code/baturalpsert/covid19-xgboost-eda

https://www.kaggle.com/code/kavishchaudhary1003/covid-19-data-classification-multiple-models

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.inspection import DecisionBoundaryDisplay	
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score, average_precision_score, confusion_matrix
from sklearn.model_selection import cross_val_score
import seaborn as sns
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import StratifiedKFold
import plotly.express as px
from tqdm import tqdm
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis, QuadraticDiscriminantAnalysis
from sklearn.utils import shuffle
from sklearn.tree import DecisionTreeClassifier
from sklearn import tree
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier, GradientBoostingClassifier 
from imblearn.over_sampling import SMOTE
from os import getcwd

In [9]:
df_covid = pd.read_csv('2_covid_death/Covid_Data.csv')
df_covid.sample(10)


,USMER,MEDICAL_UNIT,SEX,PATIENT_TYPE,DATE_DIED,INTUBED,PNEUMONIA,AGE,PREGNANT,DIABETES,...,ASTHMA,INMSUPR,HIPERTENSION,OTHER_DISEASE,CARDIOVASCULAR,OBESITY,RENAL_CHRONIC,TOBACCO,CLASIFFICATION_FINAL,ICU
883447,1,12,1,1,9999-99-99,97,2,46,2,2,...,2,2,2,2,2,2,2,2,7,97
73802,1,4,1,2,9999-99-99,2,1,57,2,2,...,2,2,2,2,2,1,2,2,3,2
944625,2,12,2,1,9999-99-99,97,2,29,97,2,...,2,2,2,2,2,2,2,2,7,97
867852,2,12,1,1,9999-99-99,97,2,29,2,2,...,2,2,2,2,2,2,2,2,7,97
1045081,1,12,1,1,9999-99-99,97,2,22,2,2,...,2,2,2,2,2,2,2,2,7,97
684952,2,12,1,1,9999-99-99,97,2,23,1,98,...,98,98,98,98,98,98,98,98,6,97
757603,1,12,2,2,9999-99-99,2,1,85,97,2,...,2,2,2,2,2,2,2,2,7,2
868521,2,12,1,1,9999-99-99,97,2,30,2,2,...,2,2,2,2,2,2,2,2,7,97
318958,2,4,2,1,9999-99-99,97,2,58,97,1,...,2,1,1,2,2,2,2,1,7,97
274439,1,4,2,1,9999-99-99,97,2,8,97,2,...,2,2,2,2,2,2,2,2,7,97


sex: female or male:

1 - female, 2 - male.

age: of the patient.

classification_final: covid test findings. 
Values 1-3 mean that the patient was diagnosed with covid in different degrees. 
4 or higher means that the patient is not a carrier of covid or that the test is inconclusive.
patient type: hospitalized or not hospitalized.

1 - hospitalized, 2 - not hospitalized.

pneumonia: whether the patient already have air sacs inflammation or not.

1 - yes, 2 - no.

pregnancy: whether the patient is pregnant or not.
1 - yes, 2 - no.
diabetes: whether the patient has diabetes or not.
1 - yes, 2 - no.
copd: Indicates whether the patient has Chronic obstructive pulmonary disease or not.
1 - yes, 2 - no.
asthma: whether the patient has asthma or not.
1 - yes, 2 - no.
inmsupr: whether the patient is immunosuppressed or not.
1 - yes, 2 - no.
hypertension: whether the patient has hypertension or not.
1 - yes, 2 - no.
cardiovascular: whether the patient has heart or blood vessels related disease.
1 - yes, 2 - no.
renal chronic: whether the patient has chronic renal disease or not.
1 - yes, 2 - no.
other disease: whether the patient has other disease or not.
1 - yes, 2 - no.
obesity: whether the patient is obese or not.
1 - yes, 2 - no.
tobacco: whether the patient is a tobacco user.
1 - yes, 2 - no.
usmr: Indicates whether the patient treated medical units of the first, second or third level.
1 - yes, 2 - no.
medical unit: type of institution of the National Health System that provided the care.
from 1 to 13.
intubed: whether the patient was connected to the ventilator.
1 - yes, 2 - no.
icu: Indicates whether the patient had been admitted to an Intensive Care Unit.
1 - yes, 2 - no.
death: indicates whether the patient died or recovered.
In the Boolean features, 1 means "yes" and 2 means "no". values as 97 and 99 are missing data.

In [3]:
df_covid.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1048575 entries, 0 to 1048574
Data columns (total 21 columns):
 #   Column                Non-Null Count    Dtype 
---  ------                --------------    ----- 
 0   USMER                 1048575 non-null  int64 
 1   MEDICAL_UNIT          1048575 non-null  int64 
 2   SEX                   1048575 non-null  int64 
 3   PATIENT_TYPE          1048575 non-null  int64 
 4   DATE_DIED             1048575 non-null  object
 5   INTUBED               1048575 non-null  int64 
 6   PNEUMONIA             1048575 non-null  int64 
 7   AGE                   1048575 non-null  int64 
 8   PREGNANT              1048575 non-null  int64 
 9   DIABETES              1048575 non-null  int64 
 10  COPD                  1048575 non-null  int64 
 11  ASTHMA                1048575 non-null  int64 
 12  INMSUPR               1048575 non-null  int64 
 13  HIPERTENSION          1048575 non-null  int64 
 14  OTHER_DISEASE         1048575 non-null  int64 
 15

In [7]:
df_covid.shape

(1048575, 21)

In [4]:
# Checking zeores and NaN in the data:
zeros_count = (df_covid == 0).sum()
print("Zero Values:", "\n")
print(zeros_count)
print("\n")
print("How many Nans:", "\n")
print(df_covid.isna().sum())

Zero Values: 

USMER                      0
MEDICAL_UNIT               0
SEX                        0
PATIENT_TYPE               0
DATE_DIED                  0
INTUBED                    0
PNEUMONIA                  0
AGE                     3862
PREGNANT                   0
DIABETES                   0
COPD                       0
ASTHMA                     0
INMSUPR                    0
HIPERTENSION               0
OTHER_DISEASE              0
CARDIOVASCULAR             0
OBESITY                    0
RENAL_CHRONIC              0
TOBACCO                    0
CLASIFFICATION_FINAL       0
ICU                        0
dtype: int64


How many Nans: 

USMER                   0
MEDICAL_UNIT            0
SEX                     0
PATIENT_TYPE            0
DATE_DIED               0
INTUBED                 0
PNEUMONIA               0
AGE                     0
PREGNANT                0
DIABETES                0
COPD                    0
ASTHMA                  0
INMSUPR                 0
HIPERT

In [26]:
# df_covid[df_covid['AGE'] == 0]
# df_covid[df_covid['AGE'] == 0].iloc[153-160]
# df_covid[df_covid['AGE'] == 0].iloc[153]
print(df_covid[df_covid['AGE'] == 0].sum())

USMER                                                                5945
MEDICAL_UNIT                                                        35854
SEX                                                                  5974
PATIENT_TYPE                                                         6355
DATE_DIED               08/05/202017/07/20209999-99-999999-99-999999-9...
INTUBED                                                            140932
PNEUMONIA                                                           10949
AGE                                                                     0
PREGNANT                                                           209127
DIABETES                                                             9493
COPD                                                                 9248
ASTHMA                                                               9232
INMSUPR                                                              9436
HIPERTENSION                          

In [25]:
# df_covid[df_covid['DATE_DIED'] == '9999-99-99']
print(df_covid[df_covid['DATE_DIED'] == '9999-99-99'].sum())


USMER                                                             1600749
MEDICAL_UNIT                                                      8877529
SEX                                                               1445604
PATIENT_TYPE                                                      1101598
DATE_DIED               9999-99-999999-99-999999-99-999999-99-999999-9...
INTUBED                                                          82402828
PNEUMONIA                                                         3338244
AGE                                                              39125515
PREGNANT                                                         47316421
DIABETES                                                          2096814
COPD                                                              2152257
ASTHMA                                                            2131574
INMSUPR                                                           2186018
HIPERTENSION                          